### Import Libraries

In [1]:
import os
import pickle
import requests

from bs4 import BeautifulSoup

### Make a Request

In [2]:
def make_request(url="https://economics.gmu.edu/course_sections",
                 headers={'User-Agent': 'Chrome/74.0.3729.169'},
                 params={'term': '201970'}):
    try:
        response = requests.get(url, headers=headers, params=params)
    except requests.exceptions.RequestException as e:
        raise SystemExit(e)

    print(response.url, response.status_code)

    return response

### Build List of Semesters Available

In [3]:
def build_semester_list():    
    #Make default request.
    response = make_request()

    #Parse resulting HTML data.
    soup = BeautifulSoup(response.text)

    #Semesters available are those available thourhg the drop-down menu. Search for drop-down menu options.
    avail_semesters = soup.find_all('select', id='term-select')    

    #Loop over each option in the drop-down menu. For each option, determine the code for the website url and its text
    #name. [1:] removes the default menu title "Choose a term".
    semesters = []

    for sem in avail_semesters[0].find_all('option')[1:]:
        semesters.append([sem['value'], sem.text])
    
    return semesters

### Download GMU Website HTML

In [9]:
def download_gmu_html(semesters):
    semester_html_data = {}

    for sem_value, sem_string in semester_list:
        params = {'term': sem_value}
        response = make_request(params=params)
        
        sem_string = sem_string.replace(' ','_')
        semester_html_data[sem_string] = {}
        semester_html_data[sem_string]['value'] = sem_value
        semester_html_data[sem_string]['url'] = response.url
        semester_html_data[sem_string]['response'] = response

    with open('html-data/semester_html_data','wb') as file:
        pickle.dump(semester_html_data, file)

    return semester_html_data

### Download Syllabi

In [5]:
def download_syllabi(directory, file, url, params):
    #Assign path, create if it does not exist.
    path = os.path.join(directory, file)

    if not os.path.exists(directory):
        os.makedirs(directory)

    #Download PDFs.
    response_pdf = make_request(url=url, params=params)

    #Write to drive.
    with open(path, "wb") as pdf:
        for chunk in response_pdf.iter_content():
            pdf.write(chunk)

### Compile Course Syllabi

In [6]:
def compile_course_syllabi(data, out_folder):
    for sem, block in data.items():
        block_bs = BeautifulSoup(block['response'].text)
        course_content = block_bs.find_all('div', class_='course content')

        for course in course_content:
            #Parse the course header for the course title--used as the output folder directory.
            course_header = course.find('header').text.split('\n')

            course_title = (course_header[2]
                            .strip()
                            .replace(':','')
                            .replace(',','')
                            .replace('\'','')
                            .replace(' ','_')
                            .replace('&',''))

            #Find the url to the course syllabus and make special arrangements for Special Topics in Economics content.
            links = course.find_all('a')

            for link in links:
                #Course titles for Special Topics in Economics
                if len(link.text.split(':')) > 1 and course_title == 'Special_Topics_in_Economics':
                        course_title = (link.text
                                        .split(':')[1]
                                        .strip()
                                        .replace(':','')
                                        .replace(',','')
                                        .replace('\'','')
                                        .replace(' ','_')
                                        .replace('&',''))
    
                #Parse URL for course syllabi links.
                if link.text == "Section Syllabus":
                    #Parse syllabi links.
                    pdf_url = 'https:{}'.format(link['href'].split("?")[0])
                    pdf_params = link['href'].split("?")[1]
                    pdf_file_name = os.path.basename(pdf_url)

                    #Download PDFs
                    out_dir = os.path.join(out_folder, course_title, sem)
                    download_syllabi(directory=out_dir, file=pdf_file_name, url=pdf_url, params=pdf_params)

In [7]:
semester_list = build_semester_list()

https://economics.gmu.edu/course_sections?term=201970 200


In [10]:
gmu_html_dict = download_gmu_html(semesters=semester_list)

https://economics.gmu.edu/course_sections?term=201970 200
https://economics.gmu.edu/course_sections?term=201940 200
https://economics.gmu.edu/course_sections?term=201910 200
https://economics.gmu.edu/course_sections?term=201870 200
https://economics.gmu.edu/course_sections?term=201840 200
https://economics.gmu.edu/course_sections?term=201810 200
https://economics.gmu.edu/course_sections?term=201770 200
https://economics.gmu.edu/course_sections?term=201740 200
https://economics.gmu.edu/course_sections?term=201710 200
https://economics.gmu.edu/course_sections?term=201670 200
https://economics.gmu.edu/course_sections?term=201640 200
https://economics.gmu.edu/course_sections?term=201610 200
https://economics.gmu.edu/course_sections?term=201570 200
https://economics.gmu.edu/course_sections?term=201540 200
https://economics.gmu.edu/course_sections?term=201512 200
https://economics.gmu.edu/course_sections?term=201510 200
https://economics.gmu.edu/course_sections?term=201472 200
https://econom

In [12]:
compile_course_syllabi(data=gmu_html_dict, out_folder='gmu-syllabi')

https://d101vc9winf8ln.cloudfront.net/syllabuses/40328/original/ECON_100_001_Alexander_Craig.pdf?1564412155 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/40337/original/ECON_105_001_Nathan_Goodman.pdf?1565106147 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/40344/original/ECON_309_003_Jason_Dunick.pdf?1567020114 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/40345/original/ECON_309_004_Peter_Jacobsen.pdf?1566841054 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/40346/original/ECON_310_002_Jay_Cochran.pdf?1564514444 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/40347/original/ECON_310_006_Nicholas_Cooper.pdf?1566849756 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/40348/original/ECON_310_007_Denver_Casey.pdf?1566847426 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/40349/original/ECON_311_001_Peter_Jacobsen.pdf?1566841076 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/40350/original/ECON_311_003_Dillon_Tauzin.pdf?1566825157 200
https:/

https://d101vc9winf8ln.cloudfront.net/syllabuses/39713/original/ECON_496_C01_Christopher_Coyne.pdf?1557928368 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/39713/original/ECON_496_C01_Christopher_Coyne.pdf?1557928368 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/39716/original/ECON_695_C02_Christopher_Coyne.pdf?1557928392 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/39716/original/ECON_695_C02_Christopher_Coyne.pdf?1557928392 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/38152/original/ECON_104_003_Thomas_Rustici.pdf?1548699896 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/38153/original/ECON_104_004_Stephen_Gillespie.pdf?1547144270 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/38154/original/ECON_105_001_Jordan_Lofthouse.pdf?1547570124 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/38159/original/ECON_308_001_Robert_Hazel.pdf?1544805435 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/38160/original/ECON_308_002_Jay_Cochran.pdf?1545

https://d101vc9winf8ln.cloudfront.net/syllabuses/36137/original/ECON_308_001_Kevin_McCabe.pdf?1535135542 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/36138/original/ECON_309_002_Jeffrey_Bridges.pdf?1533914376 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/36139/original/ECON_309_003_Jeffrey_Bridges.pdf?1533914412 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/36140/original/ECON_309_004_William_McNaught.pdf?1535376839 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/36141/original/ECON_310_002_Jay_Cochran.pdf?1535116186 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/36143/original/ECON_310_006_Charles_Hoffman.pdf?1533914646 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/36144/original/ECON_310_007_Jay_Cochran.pdf?1535116212 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/36145/original/ECON_311_001_Kaitlyn_Woltz.pdf?1535457861 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/36146/original/ECON_311_002_Garett_Jones.pdf?1535116234 200
https:/

https://d101vc9winf8ln.cloudfront.net/syllabuses/35724/original/ECON_695_C01_Christopher_Coyne.pdf?1526906796 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/35724/original/ECON_695_C01_Christopher_Coyne.pdf?1526906796 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/34017/original/ECON_103_002_Donald_Boudreaux.pdf?1515600640 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/35262/original/ECON_104_004_Stephen_Gillespie.pdf?1515089994 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/34022/original/ECON_105_001_Anthony_Good.pdf?1513629825 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/34029/original/ECON_308_001_Jay_Cochran.pdf?1515178336 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/34030/original/ECON_308_002_Jeffrey_Bridges.pdf?1513352583 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/34033/original/ECON_309_003_William_McNaught.pdf?1516195476 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/34034/original/ECON_309_004_Ennio_Piano.pdf?1515090437

https://d101vc9winf8ln.cloudfront.net/syllabuses/31994/original/ECON_311_002_Garett_Jones.pdf?1503689170 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/31995/original/ECON_311_003_Anthony_Good.pdf?1503930083 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/31996/original/ECON_321_001_Bryan_Caplan.pdf?1500645017 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/31997/original/ECON_330_001_Jeffrey_Bridges.pdf?1501777498 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/31998/original/ECON_335_001_William_McNaught.pdf?1503925644 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/31999/original/ECON_340_001_Ennio_Piano.pdf?1501085052 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/32000/original/ECON_345_001_David_Levy.pdf?1502722386 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/32001/original/ECON_345_002_Joshua_Ingber.pdf?1503923574 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/32006/original/ECON_361_001_Carrie_Meyer.pdf?1503687896 200
https://d101vc

...

https://d101vc9winf8ln.cloudfront.net/syllabuses/11924/original/ECON_308_001_Alex_Schibuola.pdf?1376509037 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/11925/original/ECON_308_002_Alex_Schibuola.pdf?1376509051 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/11926/original/ECON_309_001_Thomas_Rustici.PDF?1377541022 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/11927/original/ECON_309_002_Jason_Dunick.pdf?1377285925 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/11928/original/ECON_309_003_Jason_Dunick.pdf?1377285944 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/11942/original/ECON_321_Bryan_Caplan.pdf?1374605967 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/11943/original/ECON_330_001_Bill_McNaught.pdf?1376940447 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/11944/original/ECON_335_001_David_Chandler_Thomas.pdf?1374605981 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/11945/original/ECON_340_001_Paul_Bennett.pdf?1377095957 200
https:/

https://d101vc9winf8ln.cloudfront.net/syllabuses/11222/original/ECON_309_C01_Thomas_Rustici.PDF?1374606378 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/11223/original/ECON_310_A01_Brandon_Minster.pdf?1374606393 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/11224/original/ECON_310_C01_Philip_Wiest.pdf?1374606403 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/13180/original/ECON_335_C01_David_Thomas.pdf?1370460463 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/13181/original/ECON_345_A01_Jake_Russ.pdf?1374606437 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/11227/original/ECON_360-C01_Joshua_Trojniak.pdf?1374606453 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/11228/original/ECON_380_A01_Trey_Carson.pdf?1374606476 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/11230/original/ECON_385_Ryan_Safner.pdf?1374606493 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/11231/original/ECON_403_A01_Liya_Palagashvili.pdf?1374606514 200
https://d101vc9

https://d101vc9winf8ln.cloudfront.net/syllabuses/8063/original/412-001_Heiner.pdf?1347045907 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/8064/original/415-001_Keif.pdf?1347045921 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/9373/original/611-002_Coyne.pdf?1347045963 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/8073/original/630-001_Jones.pdf?1351867816 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/8077/original/695-002_Mitchell.pdf?1351872846 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/9389/original/695-004_McLaughlin.pdf?1351872715 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/8080/original/Williams_811-001_update.pdf?1337961346 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/8082/original/821-001_Boettke.pdf?1347046058 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/9269/original/829-001_Nye.pdf?1347046068 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/8086/original/852-002_Wagner.pdf?1344970388 200
https://d101vc9winf8l

https://d101vc9winf8ln.cloudfront.net/syllabuses/2123/original/311-002%20Schibuola.pdf?1322665104 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/2126/original/311-003%20Klee.pdf?1322665121 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/2129/original/311%20004%20Li.pdf?1322680231 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/2133/original/321-001%20Caplan.pdf?1322665302 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/4637/original/330-001%20Yoon.pdf?1322665368 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/2141/original/340%20001%20Moulton.pdf?1322680258 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/2144/original/345-001%20Al-Ubaydli.pdf?1322665672 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/2147/original/345-002%20Reynolds.pdf?1322665685 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/5548/original/345-003%20Levy.pdf?1322665701 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/2151/original/360-001%20Sabetti.pdf?1323181489 200
https

https://d101vc9winf8ln.cloudfront.net/syllabuses/2236/original/695-001_Webb.pdf?1309207107 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/2275/original/895-003_Stratmann.pdf?1309207110 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/2283/original/895-007_Storr.pdf?1309207110 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/2020/original/100-001_Boettke.pdf?1309207094 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/2043/original/104-002_Holt.pdf?1309207095 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/4635/original/104-001_Barton.pdf?1309207095 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/2080/original/308-001_Horn.pdf?1309207097 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/2083/original/309-001_Rustici.pdf?1309207098 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/2086/original/309-002_Snow.pdf?1309207098 200
https://d101vc9winf8ln.cloudfront.net/syllabuses/2122/original/311-002_Cochran.pdf?1309207101 200
https://d101vc9winf8ln.cloudfront